### Purpose

The goal of this notebook is to merge the wonderful rankings data compiled here: https://www.kaggle.com/martj42/ufc-rankings and my match data

In [1]:
import pandas as pd
from datetime import timedelta
import numpy as np

# 1. Load the Data

### Load the match data

In [2]:
#match_df = pd.read_csv("../data/ufc-master.csv")

match_df = pd.read_csv('scraped_event.csv') #We want to add ranks to the scraped event right?

#Let's put all the labels in a dataframe
match_df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = match_df['Winner'] == 'Red'
match_df['label'][mask] = 0
mask = match_df['Winner'] == 'Blue'
match_df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
match_df = match_df[(match_df['Winner'] == 'Blue') | (match_df['Winner'] == 'Red')]


#Make sure label is numeric
match_df['label'] = pd.to_numeric(match_df['label'], errors='coerce')

#Let's fix the date
match_df['date'] = pd.to_datetime(match_df['date'])

<ipython-input-2-4da9fca13556>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_df['label'][mask] = 0
<ipython-input-2-4da9fca13556>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_df['label'][mask] = 1


### Load the rankings data

In [3]:
rankings_df = pd.read_csv("../../data/rankings_history.csv")
rankings_df['date'] = pd.to_datetime(rankings_df['date'])

In [4]:
weightclass_list = rankings_df.weightclass.unique()
print(weightclass_list)

['Pound-for-Pound' 'Flyweight' 'Bantamweight' 'Featherweight'
 'Lightweight' 'Welterweight' 'Middleweight' 'Light Heavyweight'
 'Heavyweight' "Women's Bantamweight" "Women's Strawweight"
 "Women's Featherweight" "Women's Flyweight"]


The merged dataframe will contain all of the columns for the match dataframe.  It will also contain the following new columns:

* B_Pound-for-Pound_rank
* B_Flyweight_rank
* B_Bantamweight_rank
* B_Featherweight_rank
* B_Lightweight_rank
* B_Welterweight_rank
* B_Middleweight_rank
* B_Light Heavyweight_rank
* B_Heavyweight_rank
* B_Women's Bantamweight_rank
* B_Women's Strawweight_rank
* B_Women's Featherweight_rank
* B_Women's Flyweight_rank
* R_Pound-for-Pound_rank
* R_Flyweight_rank
* R_Bantamweight_rank
* R_Featherweight_rank
* R_Lightweight_rank
* R_Welterweight_rank
* R_Middleweight_rank
* R_Light Heavyweight_rank
* R_Heavyweight_rank
* R_Women's Bantamweight_rank
* R_Women's Strawweight_rank
* R_Women's Featherweight_rank
* R_Women's Flyweight_rank

* R_match_weightclass_rank
* B_match_weightclass_rank

* better_rank

The first batch of columns are just the current rank of the fighter in each weightclass.  I decided to do it this way as opposed to just having a 'rank' column and matching that to the weightclass of the fight, because a fighter can be ranked in multiple weightclasses, and that might give them an advantage that should be discoverable.  THe 'R_' or 'B_' refers to the red or blue fighter.

R_match_weightclass_rank and B_match_weightclass_rank are the rank of the fighter in the weightclass that the current match is taking place in.

better_rank will be {blue, red, neither} denoting the higher ranked fighter.


# 2. Now... How do we combine the two dataframes?

We have date information in both dataframes so I will use that.  We will get a list of all dates in the rankings dataframe.  The match dataframe will look at the most recent rankings before the date of the match and see if either fighter's name is in the list.

First let's get a list of dates from which we have ranking data


In [5]:
print(rankings_df.columns)

Index(['date', 'weightclass', 'fighter', 'rank'], dtype='object')


In [6]:
date_list = rankings_df.date.unique()
display(date_list)

array(['2013-02-04T00:00:00.000000000', '2013-02-11T00:00:00.000000000',
       '2013-02-18T00:00:00.000000000', '2013-02-25T00:00:00.000000000',
       '2013-03-04T00:00:00.000000000', '2013-03-18T00:00:00.000000000',
       '2013-04-08T00:00:00.000000000', '2013-04-15T00:00:00.000000000',
       '2013-04-22T00:00:00.000000000', '2013-04-29T00:00:00.000000000',
       '2013-05-28T00:00:00.000000000', '2013-06-10T00:00:00.000000000',
       '2013-06-18T00:00:00.000000000', '2013-07-08T00:00:00.000000000',
       '2013-07-29T00:00:00.000000000', '2013-08-05T00:00:00.000000000',
       '2013-08-19T00:00:00.000000000', '2013-08-30T00:00:00.000000000',
       '2013-09-02T00:00:00.000000000', '2013-09-06T00:00:00.000000000',
       '2013-09-23T00:00:00.000000000', '2013-10-11T00:00:00.000000000',
       '2013-10-21T00:00:00.000000000', '2013-10-28T00:00:00.000000000',
       '2013-11-08T00:00:00.000000000', '2013-11-11T00:00:00.000000000',
       '2013-11-18T00:00:00.000000000', '2013-12-03

In [7]:
print(min(date_list))

2013-02-04T00:00:00.000000000


In [8]:
max_date = max(date_list)
print(max_date)

2021-04-26T00:00:00.000000000


We have matchup data that goes back a few years earlier than the ranking data, but that isn't a big deal.  We just have to write code that won't return an error if it can't find appropriate ranking data

Let's try to look smart and see if we can figure this out using a lambda function

In [9]:
display(rankings_df.head())

date      weightclass            fighter  rank
0 2013-02-04  Pound-for-Pound     Anderson Silva     1
1 2013-02-04  Pound-for-Pound          Jon Jones     2
2 2013-02-04  Pound-for-Pound  Georges St-Pierre     3
3 2013-02-04  Pound-for-Pound          Jose Aldo     4
4 2013-02-04  Pound-for-Pound   Benson Henderson     5

In [10]:
display(match_df.columns)

Index(['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev', 'date',
       'location', 'country', 'Winner',
       ...
       'B_td_attempted_bout', 'R_td_pct_bout', 'B_td_pct_bout',
       'R_sub_attempts_bout', 'B_sub_attempts_bout', 'R_pass_bout',
       'B_pass_bout', 'R_rev_bout', 'B_rev_bout', 'label'],
      dtype='object', length=138)

In [11]:
test_date_list = match_df.date.unique()
display(test_date_list)


array(['2021-02-27T00:00:00.000000000'], dtype='datetime64[ns]')

In [12]:
def return_rank(fighter_name, date, wc):
    rank = ''
    keep_going = True;
    previous_d = max_date
    #We need to add something so this works for upcoming events
    for d in date_list:
        if keep_going:
            time_dif =  (d - date).total_seconds()
            print(time_dif)
            if ((time_dif > -1) or ((max_date - date).total_seconds() < 0)):
                
                keep_going = False
                #print(fighter_name, time_dif, date, wc, d)
                temp_rankings_df = rankings_df[rankings_df['date']==previous_d].copy()
                temp_rankings_df = temp_rankings_df[temp_rankings_df['weightclass']==wc]
                temp_rankings_df = temp_rankings_df[temp_rankings_df['fighter']==fighter_name]
                #This means we have a match.  We need to return the rank
                if len(temp_rankings_df) > 0:
                    rank = int(temp_rankings_df.iloc[0]['rank'])
                    #display(rank)
                    #print(fighter_name)
                #print(len(temp_rankings_df))
            else:
                previous_d = d
    if isinstance(rank, int):
        print(rank)
        return(rank)
    else:
        return('')

better_rank

In [13]:
match_df['B_match_weightclass_rank'] = match_df.apply(lambda x: return_rank(x['B_fighter'],
                                                                         x['date'],
                                                                         x['weight_class']),axis=1)

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-139968000.0
-139536000.0
-138758400.0
-138153600.0
-135907200.0
-135129600.0
-134697600.0
-133920000.0
-133488000.0
-132710400.0
-132278400.0
-131068800.0
-129686400.0
-128476800.0
-127872000.0
-127440000.0
-126748800.0
-125625600.0
-124848000.0
-124243200.0
-122428800.0
-121824000.0
-121219200.0
-119404800.0
-118195200.0
-117590400.0
-116985600.0
-116380800.0
-115776000.0
-114739200.0
-114048000.0
-113356800.0
-112752000.0
-112320000.0
-109900800.0
-109123200.0
-108518400.0
-108086400.0
-106876800.0
-105667200.0
-105062400.0
-104284800.0
-103852800.0
-103248000.0
-102470400.0
-101865600.0
-101260800.0
-100828800.0
-99619200.0
-98236800.0
-97632000.0
-97200000.0
-96422400.0
-95990400.0
-95212800.0
-94780800.0
-94176000.0
-92793600.0
-90979200.0
-90374400.0
-89942400.0
-87955200.0
-87350400.0
-86745600.0
-86140800.0
-85536000.0
-84412800.0
-83116800.0
-82684800.0
-81907200.0
-81302400.0
-80870400.0
-78883200.0
-77673600.0
-77068800.0
-76636800.0
-75427200.0
-73440000.0
-72835200.0
-724

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0
-193968000.0
-192672000.0
-192153600.0
-191548800.0
-190339200.0
-189648000.0
-189129600.0
-187920000.0
-187315200.0
-186105600.0
-185500800.0
-184896000.0
-184291200.0
-183081600.0
-181872000.0
-181267200.0
-180662400.0
-180057600.0
-179452800.0
-178848000.0
-177638400.0
-177033600.0
-176428800.0
-175824000.0
-175219200.0
-173923200.0
-172800000.0
-170985600.0
-170380800.0
-168566400.0
-167356800.0
-166752000.0
-166147200.0
-165542400.0
-164332800.0
-163728000.0
-162518400.0
-161222400.0
-160099200.0
-159494400.0
-158198400.0
-157680000.0
-157075200.0
-155865600.0
-153964800.0
-153446400.0

In [14]:
match_df['R_match_weightclass_rank'] = match_df.apply(lambda x: return_rank(x['R_fighter'],
                                                                         x['date'],
                                                                         x['weight_class']),axis=1)

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0
-193968000.0
-192672000.0
-192153600.0
-191548800.0
-190339200.0
-189648000.0
-189129600.0
-187920000.0
-187315200.0
-186105600.0
-185500800.0
-184896000.0
-184291200.0
-183081600.0
-181872000.0
-181267200.0
-180662400.0
-180057600.0
-179452800.0
-178848000.0
-177638400.0
-177033600.0
-176428800.0
-175824000.0
-175219200.0
-173923200.0
-172800000.0

-196387200.0
-195782400.0
-195177600.0
-193968000.0
-192672000.0
-192153600.0
-191548800.0
-190339200.0
-189648000.0
-189129600.0
-187920000.0
-187315200.0
-186105600.0
-185500800.0
-184896000.0
-184291200.0
-183081600.0
-181872000.0
-181267200.0
-180662400.0
-180057600.0
-179452800.0
-178848000.0
-177638400.0
-177033600.0
-176428800.0
-175824000.0
-175219200.0
-173923200.0
-172800000.0
-170985600.0
-170380800.0
-168566400.0
-167356800.0
-166752000.0
-166147200.0
-165542400.0
-164332800.0
-163728000.0
-162518400.0
-161222400.0
-160099200.0
-159494400.0
-158198400.0
-157680000.0
-157075200.0
-155865600.0
-153964800.0
-153446400.0
-152841600.0
-151545600.0
-151027200.0
-149731200.0
-149212800.0
-148003200.0
-146188800.0
-145843200.0
-144806400.0
-144201600.0
-143683200.0
-142387200.0
-141782400.0
-141350400.0
-140572800.0
-139968000.0
-139536000.0
-138758400.0
-138153600.0
-135907200.0
-135129600.0
-134697600.0
-133920000.0
-133488000.0
-132710400.0
-132278400.0
-131068800.0
-129686400.0

In [15]:
match_df['R_Women\'s Flyweight_rank'] = match_df.apply(lambda x: return_rank(x['R_fighter'],
                                                                         x['date'],
                                                                         'Women\'s Flyweight'),axis=1)

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-192153600.0
-191548800.0
-190339200.0
-189648000.0
-189129600.0
-187920000.0
-187315200.0
-186105600.0
-185500800.0
-184896000.0
-184291200.0
-183081600.0
-181872000.0
-181267200.0
-180662400.0
-180057600.0
-179452800.0
-178848000.0
-177638400.0
-177033600.0
-176428800.0
-175824000.0
-175219200.0
-173923200.0
-172800000.0
-170985600.0
-170380800.0
-168566400.0
-167356800.0
-166752000.0
-166147200.0
-165542400.0
-164332800.0
-163728000.0
-162518400.0
-161222400.0
-160099200.0
-159494400.0
-158198400.0
-157680000.0
-157075200.0
-155865600.0
-153964800.0
-153446400.0
-152841600.0
-151545600.0
-151027200.0
-149731200.0
-149212800.0
-148003200.0
-146188800.0
-145843200.0
-144806400.0
-144201600.0
-143683200.0
-142387200.0
-141782400.0
-141350400.0
-140572800.0
-139968000.0
-139536000.0
-138758400.0
-138153600.0
-135907200.0
-135129600.0
-134697600.0
-133920000.0
-133488000.0
-132710400.0
-132278400.0
-131068800.0
-129686400.0
-128476800.0
-127872000.0
-127440000.0
-126748800.0
-125625600.0

In [16]:
match_df['R_Women\'s Featherweight_rank'] = match_df.apply(lambda x: return_rank(x['R_fighter'],
                                                                         x['date'],
                                                                         'Women\'s Featherweight'),axis=1)

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-143683200.0
-142387200.0
-141782400.0
-141350400.0
-140572800.0
-139968000.0
-139536000.0
-138758400.0
-138153600.0
-135907200.0
-135129600.0
-134697600.0
-133920000.0
-133488000.0
-132710400.0
-132278400.0
-131068800.0
-129686400.0
-128476800.0
-127872000.0
-127440000.0
-126748800.0
-125625600.0
-124848000.0
-124243200.0
-122428800.0
-121824000.0
-121219200.0
-119404800.0
-118195200.0
-117590400.0
-116985600.0
-116380800.0
-115776000.0
-114739200.0
-114048000.0
-113356800.0
-112752000.0
-112320000.0
-109900800.0
-109123200.0
-108518400.0
-108086400.0
-106876800.0
-105667200.0
-105062400.0
-104284800.0
-103852800.0
-103248000.0
-102470400.0
-101865600.0
-101260800.0
-100828800.0
-99619200.0
-98236800.0
-97632000.0
-97200000.0
-96422400.0
-95990400.0
-95212800.0
-94780800.0
-94176000.0
-92793600.0
-90979200.0
-90374400.0
-89942400.0
-87955200.0
-87350400.0
-86745600.0
-86140800.0
-85536000.0
-84412800.0
-83116800.0
-82684800.0
-81907200.0
-81302400.0
-80870400.0
-78883200.0
-77673600.0

-149212800.0
-148003200.0
-146188800.0
-145843200.0
-144806400.0
-144201600.0
-143683200.0
-142387200.0
-141782400.0
-141350400.0
-140572800.0
-139968000.0
-139536000.0
-138758400.0
-138153600.0
-135907200.0
-135129600.0
-134697600.0
-133920000.0
-133488000.0
-132710400.0
-132278400.0
-131068800.0
-129686400.0
-128476800.0
-127872000.0
-127440000.0
-126748800.0
-125625600.0
-124848000.0
-124243200.0
-122428800.0
-121824000.0
-121219200.0
-119404800.0
-118195200.0
-117590400.0
-116985600.0
-116380800.0
-115776000.0
-114739200.0
-114048000.0
-113356800.0
-112752000.0
-112320000.0
-109900800.0
-109123200.0
-108518400.0
-108086400.0
-106876800.0
-105667200.0
-105062400.0
-104284800.0
-103852800.0
-103248000.0
-102470400.0
-101865600.0
-101260800.0
-100828800.0
-99619200.0
-98236800.0
-97632000.0
-97200000.0
-96422400.0
-95990400.0
-95212800.0
-94780800.0
-94176000.0
-92793600.0
-90979200.0
-90374400.0
-89942400.0
-87955200.0
-87350400.0
-86745600.0
-86140800.0
-85536000.0
-84412800.0
-8311

In [17]:
match_df['R_Women\'s Strawweight_rank'] = match_df.apply(lambda x: return_rank(x['R_fighter'],
                                                                         x['date'],
                                                                         'Women\'s Strawweight'),axis=1)

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-133920000.0
-133488000.0
-132710400.0
-132278400.0
-131068800.0
-129686400.0
-128476800.0
-127872000.0
-127440000.0
-126748800.0
-125625600.0
-124848000.0
-124243200.0
-122428800.0
-121824000.0
-121219200.0
-119404800.0
-118195200.0
-117590400.0
-116985600.0
-116380800.0
-115776000.0
-114739200.0
-114048000.0
-113356800.0
-112752000.0
-112320000.0
-109900800.0
-109123200.0
-108518400.0
-108086400.0
-106876800.0
-105667200.0
-105062400.0
-104284800.0
-103852800.0
-103248000.0
-102470400.0
-101865600.0
-101260800.0
-100828800.0
-99619200.0
-98236800.0
-97632000.0
-97200000.0
-96422400.0
-95990400.0
-95212800.0
-94780800.0
-94176000.0
-92793600.0
-90979200.0
-90374400.0
-89942400.0
-87955200.0
-87350400.0
-86745600.0
-86140800.0
-85536000.0
-84412800.0
-83116800.0
-82684800.0
-81907200.0
-81302400.0
-80870400.0
-78883200.0
-77673600.0
-77068800.0
-76636800.0
-75427200.0
-73440000.0
-72835200.0
-72403200.0
-71798400.0
-71020800.0
-70588800.0
-69811200.0
-69379200.0
-68083200.0
-66355200.0

-78883200.0
-77673600.0
-77068800.0
-76636800.0
-75427200.0
-73440000.0
-72835200.0
-72403200.0
-71798400.0
-71020800.0
-70588800.0
-69811200.0
-69379200.0
-68083200.0
-66355200.0
-65145600.0
-64540800.0
-63849600.0
-63331200.0
-62640000.0
-62035200.0
-61344000.0
-60825600.0
-60307200.0
-60134400.0
-59097600.0
-58492800.0
-57801600.0
-57283200.0
-56678400.0
-55987200.0
-54864000.0
-51840000.0
-51235200.0
-50630400.0
-50025600.0
-49420800.0
-48816000.0
-48211200.0
-47001600.0
-46396800.0
-45792000.0
-45187200.0
-44582400.0
-43977600.0
-43372800.0
-42768000.0
-42163200.0
-41558400.0
-40953600.0
-40348800.0
-38448000.0
-37929600.0
-37238400.0
-34905600.0
-34300800.0
-33091200.0
-32486400.0
-31881600.0
-31276800.0
-30672000.0
-30067200.0
-25228800.0
-24537600.0
-23328000.0
-22809600.0
-22204800.0
-21600000.0
-20995200.0
-19699200.0
-18576000.0
-17971200.0
-17366400.0
-16761600.0
-16156800.0
-15552000.0
-14947200.0
-14256000.0
-13737600.0
-13132800.0
-12528000.0
-11923200.0
-11318400.0
-107

In [18]:
match_df['R_Women\'s Bantamweight_rank'] = match_df.apply(lambda x: return_rank(x['R_fighter'],
                                                                         x['date'],
                                                                         'Women\'s Bantamweight'),axis=1)

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-1036800.0
-432000.0
172800.0
-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-1963872

-170380800.0
-168566400.0
-167356800.0
-166752000.0
-166147200.0
-165542400.0
-164332800.0
-163728000.0
-162518400.0
-161222400.0
-160099200.0
-159494400.0
-158198400.0
-157680000.0
-157075200.0
-155865600.0
-153964800.0
-153446400.0
-152841600.0
-151545600.0
-151027200.0
-149731200.0
-149212800.0
-148003200.0
-146188800.0
-145843200.0
-144806400.0
-144201600.0
-143683200.0
-142387200.0
-141782400.0
-141350400.0
-140572800.0
-139968000.0
-139536000.0
-138758400.0
-138153600.0
-135907200.0
-135129600.0
-134697600.0
-133920000.0
-133488000.0
-132710400.0
-132278400.0
-131068800.0
-129686400.0
-128476800.0
-127872000.0
-127440000.0
-126748800.0
-125625600.0
-124848000.0
-124243200.0
-122428800.0
-121824000.0
-121219200.0
-119404800.0
-118195200.0
-117590400.0
-116985600.0
-116380800.0
-115776000.0
-114739200.0
-114048000.0
-113356800.0
-112752000.0
-112320000.0
-109900800.0
-109123200.0
-108518400.0
-108086400.0
-106876800.0
-105667200.0
-105062400.0
-104284800.0
-103852800.0
-103248000.0

-18576000.0
-17971200.0
-17366400.0
-16761600.0
-16156800.0
-15552000.0
-14947200.0
-14256000.0
-13737600.0
-13132800.0
-12528000.0
-11923200.0
-11318400.0
-10713600.0
-10108800.0
-9504000.0
-8899200.0
-8294400.0
-7689600.0
-7084800.0
-6480000.0
-5875200.0
-3456000.0
-2851200.0
-1641600.0
-1036800.0
-432000.0
172800.0


In [19]:
match_df['R_Heavyweight_rank'] = match_df.apply(lambda x: return_rank(x['R_fighter'],
                                                                         x['date'],
                                                                         'Heavyweight'),axis=1)

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-63849600.0
-63331200.0
-62640000.0
-62035200.0
-61344000.0
-60825600.0
-60307200.0
-60134400.0
-59097600.0
-58492800.0
-57801600.0
-57283200.0
-56678400.0
-55987200.0
-54864000.0
-51840000.0
-51235200.0
-50630400.0
-50025600.0
-49420800.0
-48816000.0
-48211200.0
-47001600.0
-46396800.0
-45792000.0
-45187200.0
-44582400.0
-43977600.0
-43372800.0
-42768000.0
-42163200.0
-41558400.0
-40953600.0
-40348800.0
-38448000.0
-37929600.0
-37238400.0
-34905600.0
-34300800.0
-33091200.0
-32486400.0
-31881600.0
-31276800.0
-30672000.0
-30067200.0
-25228800.0
-24537600.0
-23328000.0
-22809600.0
-22204800.0
-21600000.0
-20995200.0
-19699200.0
-18576000.0
-17971200.0
-17366400.0
-16761600.0
-16156800.0
-15552000.0
-14947200.0
-14256000.0
-13737600.0
-13132800.0
-12528000.0
-11923200.0
-11318400.0
-10713600.0
-10108800.0
-9504000.0
-8899200.0
-8294400.0
-7689600.0
-7084800.0
-6480000.0
-5875200.0
-3456000.0
-2851200.0
-1641600.0
-1036800.0
-432000.0
172800.0
-254448000.0
-253843200.0
-253238400.0
-2526

-164332800.0
-163728000.0
-162518400.0
-161222400.0
-160099200.0
-159494400.0
-158198400.0
-157680000.0
-157075200.0
-155865600.0
-153964800.0
-153446400.0
-152841600.0
-151545600.0
-151027200.0
-149731200.0
-149212800.0
-148003200.0
-146188800.0
-145843200.0
-144806400.0
-144201600.0
-143683200.0
-142387200.0
-141782400.0
-141350400.0
-140572800.0
-139968000.0
-139536000.0
-138758400.0
-138153600.0
-135907200.0
-135129600.0
-134697600.0
-133920000.0
-133488000.0
-132710400.0
-132278400.0
-131068800.0
-129686400.0
-128476800.0
-127872000.0
-127440000.0
-126748800.0
-125625600.0
-124848000.0
-124243200.0
-122428800.0
-121824000.0
-121219200.0
-119404800.0
-118195200.0
-117590400.0
-116985600.0
-116380800.0
-115776000.0
-114739200.0
-114048000.0
-113356800.0
-112752000.0
-112320000.0
-109900800.0
-109123200.0
-108518400.0
-108086400.0
-106876800.0
-105667200.0
-105062400.0
-104284800.0
-103852800.0
-103248000.0
-102470400.0
-101865600.0
-101260800.0
-100828800.0
-99619200.0
-98236800.0
-

In [20]:
match_df['R_Light Heavyweight_rank'] = match_df.apply(lambda x: return_rank(x['R_fighter'],
                                                                         x['date'],
                                                                         'Light Heavyweight'),axis=1)

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0
-193968000.0
-192672000.0
-192153600.0
-191548800.0
-190339200.0
-189648000.0
-189129600.0
-187920000.0
-187315200.0
-186105600.0
-185500800.0
-184896000.0
-184291200.0
-183081600.0
-181872000.0
-181267200.0
-180662400.0
-180057600.0
-179452800.0
-178848000.0
-177638400.0
-177033600.0
-176428800.0
-175824000.0
-175219200.0
-173923200.0
-172800000.0
-170985600.0
-170380800.0
-168566400.0
-167356800.0
-166752000.0
-166147200.0
-165542400.0
-164332800.0
-163728000.0
-162518400.0
-161222400.0

-57283200.0
-56678400.0
-55987200.0
-54864000.0
-51840000.0
-51235200.0
-50630400.0
-50025600.0
-49420800.0
-48816000.0
-48211200.0
-47001600.0
-46396800.0
-45792000.0
-45187200.0
-44582400.0
-43977600.0
-43372800.0
-42768000.0
-42163200.0
-41558400.0
-40953600.0
-40348800.0
-38448000.0
-37929600.0
-37238400.0
-34905600.0
-34300800.0
-33091200.0
-32486400.0
-31881600.0
-31276800.0
-30672000.0
-30067200.0
-25228800.0
-24537600.0
-23328000.0
-22809600.0
-22204800.0
-21600000.0
-20995200.0
-19699200.0
-18576000.0
-17971200.0
-17366400.0
-16761600.0
-16156800.0
-15552000.0
-14947200.0
-14256000.0
-13737600.0
-13132800.0
-12528000.0
-11923200.0
-11318400.0
-10713600.0
-10108800.0
-9504000.0
-8899200.0
-8294400.0
-7689600.0
-7084800.0
-6480000.0
-5875200.0
-3456000.0
-2851200.0
-1641600.0
-1036800.0
-432000.0
172800.0
-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142

In [21]:
match_df['R_Middleweight_rank'] = match_df.apply(lambda x: return_rank(x['R_fighter'],
                                                                         x['date'],
                                                                         'Middleweight'),axis=1)

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0
-193968000.0
-192672000.0
-192153600.0
-191548800.0
-190339200.0
-189648000.0
-189129600.0
-187920000.0
-187315200.0
-186105600.0
-185500800.0
-184896000.0
-184291200.0
-183081600.0
-181872000.0
-181267200.0
-180662400.0
-180057600.0
-179452800.0
-178848000.0
-177638400.0
-177033600.0
-176428800.0
-175824000.0
-175219200.0
-173923200.0
-172800000.0
-170985600.0
-170380800.0
-168566400.0
-167356800.0
-166752000.0
-166147200.0
-165542400.0
-164332800.0
-163728000.0
-162518400.0
-161222400.0
-160099200.0
-159494400.0
-158198400.0
-157680000.0
-157075200.0
-155865600.0
-153964800.0
-153446400.0
-152841600.0
-151545600.0
-151027200.0
-149731200.0
-149212800.0
-148003200.0
-146188800.0
-145843200.0
-144806400.0
-144201600.0
-143683200.0
-142387200.0
-141782400.0
-141350400.0
-140572800.0
-139968000.0
-139536000.0
-138758400.0
-138153600.0
-135907200.0
-135129600.0
-134697600.0
-133920000.0

In [22]:
match_df['R_Welterweight_rank'] = match_df.apply(lambda x: return_rank(x['R_fighter'],
                                                                         x['date'],
                                                                         'Welterweight'),axis=1)

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0
-193968000.0
-192672000.0
-192153600.0
-191548800.0
-190339200.0
-189648000.0
-189129600.0
-187920000.0
-187315200.0
-186105600.0
-185500800.0
-184896000.0
-184291200.0
-183081600.0
-181872000.0
-181267200.0
-180662400.0
-180057600.0
-179452800.0
-178848000.0
-177638400.0
-177033600.0
-176428800.0
-175824000.0
-175219200.0
-173923200.0
-172800000.0
-170985600.0
-170380800.0
-168566400.0
-167356800.0
-166752000.0
-166147200.0
-165542400.0
-164332800.0
-163728000.0
-162518400.0
-161222400.0
-160099200.0
-159494400.0
-158198400.0
-157680000.0
-157075200.0
-155865600.0
-153964800.0
-153446400.0
-152841600.0
-151545600.0
-151027200.0
-149731200.0
-149212800.0
-148003200.0
-146188800.0
-145843200.0

-31881600.0
-31276800.0
-30672000.0
-30067200.0
-25228800.0
-24537600.0
-23328000.0
-22809600.0
-22204800.0
-21600000.0
-20995200.0
-19699200.0
-18576000.0
-17971200.0
-17366400.0
-16761600.0
-16156800.0
-15552000.0
-14947200.0
-14256000.0
-13737600.0
-13132800.0
-12528000.0
-11923200.0
-11318400.0
-10713600.0
-10108800.0
-9504000.0
-8899200.0
-8294400.0
-7689600.0
-7084800.0
-6480000.0
-5875200.0
-3456000.0
-2851200.0
-1641600.0
-1036800.0
-432000.0
172800.0
-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-21

In [23]:
match_df['R_Lightweight_rank'] = match_df.apply(lambda x: return_rank(x['R_fighter'],
                                                                         x['date'],
                                                                         'Lightweight'),axis=1)

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-58492800.0
-57801600.0
-57283200.0
-56678400.0
-55987200.0
-54864000.0
-51840000.0
-51235200.0
-50630400.0
-50025600.0
-49420800.0
-48816000.0
-48211200.0
-47001600.0
-46396800.0
-45792000.0
-45187200.0
-44582400.0
-43977600.0
-43372800.0
-42768000.0
-42163200.0
-41558400.0
-40953600.0
-40348800.0
-38448000.0
-37929600.0
-37238400.0
-34905600.0
-34300800.0
-33091200.0
-32486400.0
-31881600.0
-31276800.0
-30672000.0
-30067200.0
-25228800.0
-24537600.0
-23328000.0
-22809600.0
-22204800.0
-21600000.0
-20995200.0
-19699200.0
-18576000.0
-17971200.0
-17366400.0
-16761600.0
-16156800.0
-15552000.0
-14947200.0
-14256000.0
-13737600.0
-13132800.0
-12528000.0
-11923200.0
-11318400.0
-10713600.0
-10108800.0
-9504000.0
-8899200.0
-8294400.0
-7689600.0
-7084800.0
-6480000.0
-5875200.0
-3456000.0
-2851200.0
-1641600.0
-1036800.0
-432000.0
172800.0
-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-24356160

-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0
-193968000.0
-192672000.0
-192153600.0
-191548800.0
-190339200.0
-189648000.0
-189129600.0
-187920000.0
-187315200.0
-186105600.0
-185500800.0
-184896000.0
-184291200.0
-183081600.0
-181872000.0
-181267200.0
-180662400.0
-180057600.0
-179452800.0
-178848000.0
-177638400.0
-177033600.0
-176428800.0
-175824000.0
-175219200.0
-173923200.0
-172800000.0
-170985600.0
-170380800.0
-168566400.0
-167356800.0
-166752000.0
-166147200.0
-165542400.0
-164332800.0

-40348800.0
-38448000.0
-37929600.0
-37238400.0
-34905600.0
-34300800.0
-33091200.0
-32486400.0
-31881600.0
-31276800.0
-30672000.0
-30067200.0
-25228800.0
-24537600.0
-23328000.0
-22809600.0
-22204800.0
-21600000.0
-20995200.0
-19699200.0
-18576000.0
-17971200.0
-17366400.0
-16761600.0
-16156800.0
-15552000.0
-14947200.0
-14256000.0
-13737600.0
-13132800.0
-12528000.0
-11923200.0
-11318400.0
-10713600.0
-10108800.0
-9504000.0
-8899200.0
-8294400.0
-7689600.0
-7084800.0
-6480000.0
-5875200.0
-3456000.0
-2851200.0
-1641600.0
-1036800.0
-432000.0
172800.0


In [24]:
match_df['R_Featherweight_rank'] = match_df.apply(lambda x: return_rank(x['R_fighter'],
                                                                         x['date'],
                                                                         'Featherweight'),axis=1)

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-127872000.0
-127440000.0
-126748800.0
-125625600.0
-124848000.0
-124243200.0
-122428800.0
-121824000.0
-121219200.0
-119404800.0
-118195200.0
-117590400.0
-116985600.0
-116380800.0
-115776000.0
-114739200.0
-114048000.0
-113356800.0
-112752000.0
-112320000.0
-109900800.0
-109123200.0
-108518400.0
-108086400.0
-106876800.0
-105667200.0
-105062400.0
-104284800.0
-103852800.0
-103248000.0
-102470400.0
-101865600.0
-101260800.0
-100828800.0
-99619200.0
-98236800.0
-97632000.0
-97200000.0
-96422400.0
-95990400.0
-95212800.0
-94780800.0
-94176000.0
-92793600.0
-90979200.0
-90374400.0
-89942400.0
-87955200.0
-87350400.0
-86745600.0
-86140800.0
-85536000.0
-84412800.0
-83116800.0
-82684800.0
-81907200.0
-81302400.0
-80870400.0
-78883200.0
-77673600.0
-77068800.0
-76636800.0
-75427200.0
-73440000.0
-72835200.0
-72403200.0
-71798400.0
-71020800.0
-70588800.0
-69811200.0
-69379200.0
-68083200.0
-66355200.0
-65145600.0
-64540800.0
-63849600.0
-63331200.0
-62640000.0
-62035200.0
-61344000.0
-60825

-42768000.0
-42163200.0
-41558400.0
-40953600.0
-40348800.0
-38448000.0
-37929600.0
-37238400.0
-34905600.0
-34300800.0
-33091200.0
-32486400.0
-31881600.0
-31276800.0
-30672000.0
-30067200.0
-25228800.0
-24537600.0
-23328000.0
-22809600.0
-22204800.0
-21600000.0
-20995200.0
-19699200.0
-18576000.0
-17971200.0
-17366400.0
-16761600.0
-16156800.0
-15552000.0
-14947200.0
-14256000.0
-13737600.0
-13132800.0
-12528000.0
-11923200.0
-11318400.0
-10713600.0
-10108800.0
-9504000.0
-8899200.0
-8294400.0
-7689600.0
-7084800.0
-6480000.0
-5875200.0
-3456000.0
-2851200.0
-1641600.0
-1036800.0
-432000.0
172800.0
-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-2

-151545600.0
-151027200.0
-149731200.0
-149212800.0
-148003200.0
-146188800.0
-145843200.0
-144806400.0
-144201600.0
-143683200.0
-142387200.0
-141782400.0
-141350400.0
-140572800.0
-139968000.0
-139536000.0
-138758400.0
-138153600.0
-135907200.0
-135129600.0
-134697600.0
-133920000.0
-133488000.0
-132710400.0
-132278400.0
-131068800.0
-129686400.0
-128476800.0
-127872000.0
-127440000.0
-126748800.0
-125625600.0
-124848000.0
-124243200.0
-122428800.0
-121824000.0
-121219200.0
-119404800.0
-118195200.0
-117590400.0
-116985600.0
-116380800.0
-115776000.0
-114739200.0
-114048000.0
-113356800.0
-112752000.0
-112320000.0
-109900800.0
-109123200.0
-108518400.0
-108086400.0
-106876800.0
-105667200.0
-105062400.0
-104284800.0
-103852800.0
-103248000.0
-102470400.0
-101865600.0
-101260800.0
-100828800.0
-99619200.0
-98236800.0
-97632000.0
-97200000.0
-96422400.0
-95990400.0
-95212800.0
-94780800.0
-94176000.0
-92793600.0
-90979200.0
-90374400.0
-89942400.0
-87955200.0
-87350400.0
-86745600.0
-8

In [25]:
match_df['R_Bantamweight_rank'] = match_df.apply(lambda x: return_rank(x['R_fighter'],
                                                                         x['date'],
                                                                         'Bantamweight'),axis=1)

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-126748800.0
-125625600.0
-124848000.0
-124243200.0
-122428800.0
-121824000.0
-121219200.0
-119404800.0
-118195200.0
-117590400.0
-116985600.0
-116380800.0
-115776000.0
-114739200.0
-114048000.0
-113356800.0
-112752000.0
-112320000.0
-109900800.0
-109123200.0
-108518400.0
-108086400.0
-106876800.0
-105667200.0
-105062400.0
-104284800.0
-103852800.0
-103248000.0
-102470400.0
-101865600.0
-101260800.0
-100828800.0
-99619200.0
-98236800.0
-97632000.0
-97200000.0
-96422400.0
-95990400.0
-95212800.0
-94780800.0
-94176000.0
-92793600.0
-90979200.0
-90374400.0
-89942400.0
-87955200.0
-87350400.0
-86745600.0
-86140800.0
-85536000.0
-84412800.0
-83116800.0
-82684800.0
-81907200.0
-81302400.0
-80870400.0
-78883200.0
-77673600.0
-77068800.0
-76636800.0
-75427200.0
-73440000.0
-72835200.0
-72403200.0
-71798400.0
-71020800.0
-70588800.0
-69811200.0
-69379200.0
-68083200.0
-66355200.0
-65145600.0
-64540800.0
-63849600.0
-63331200.0
-62640000.0
-62035200.0
-61344000.0
-60825600.0
-60307200.0
-6013440

-95212800.0
-94780800.0
-94176000.0
-92793600.0
-90979200.0
-90374400.0
-89942400.0
-87955200.0
-87350400.0
-86745600.0
-86140800.0
-85536000.0
-84412800.0
-83116800.0
-82684800.0
-81907200.0
-81302400.0
-80870400.0
-78883200.0
-77673600.0
-77068800.0
-76636800.0
-75427200.0
-73440000.0
-72835200.0
-72403200.0
-71798400.0
-71020800.0
-70588800.0
-69811200.0
-69379200.0
-68083200.0
-66355200.0
-65145600.0
-64540800.0
-63849600.0
-63331200.0
-62640000.0
-62035200.0
-61344000.0
-60825600.0
-60307200.0
-60134400.0
-59097600.0
-58492800.0
-57801600.0
-57283200.0
-56678400.0
-55987200.0
-54864000.0
-51840000.0
-51235200.0
-50630400.0
-50025600.0
-49420800.0
-48816000.0
-48211200.0
-47001600.0
-46396800.0
-45792000.0
-45187200.0
-44582400.0
-43977600.0
-43372800.0
-42768000.0
-42163200.0
-41558400.0
-40953600.0
-40348800.0
-38448000.0
-37929600.0
-37238400.0
-34905600.0
-34300800.0
-33091200.0
-32486400.0
-31881600.0
-31276800.0
-30672000.0
-30067200.0
-25228800.0
-24537600.0
-23328000.0
-228

In [26]:
match_df['R_Flyweight_rank'] = match_df.apply(lambda x: return_rank(x['R_fighter'],
                                                                         x['date'],
                                                                         'Flyweight'),axis=1)

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-181872000.0
-181267200.0
-180662400.0
-180057600.0
-179452800.0
-178848000.0
-177638400.0
-177033600.0
-176428800.0
-175824000.0
-175219200.0
-173923200.0
-172800000.0
-170985600.0
-170380800.0
-168566400.0
-167356800.0
-166752000.0
-166147200.0
-165542400.0
-164332800.0
-163728000.0
-162518400.0
-161222400.0
-160099200.0
-159494400.0
-158198400.0
-157680000.0
-157075200.0
-155865600.0
-153964800.0
-153446400.0
-152841600.0
-151545600.0
-151027200.0
-149731200.0
-149212800.0
-148003200.0
-146188800.0
-145843200.0
-144806400.0
-144201600.0
-143683200.0
-142387200.0
-141782400.0
-141350400.0
-140572800.0
-139968000.0
-139536000.0
-138758400.0
-138153600.0
-135907200.0
-135129600.0
-134697600.0
-133920000.0
-133488000.0
-132710400.0
-132278400.0
-131068800.0
-129686400.0
-128476800.0
-127872000.0
-127440000.0
-126748800.0
-125625600.0
-124848000.0
-124243200.0
-122428800.0
-121824000.0
-121219200.0
-119404800.0
-118195200.0
-117590400.0
-116985600.0
-116380800.0
-115776000.0
-114739200.0

-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0
-193968000.0
-192672000.0
-192153600.0
-191548800.0
-190339200.0
-189648000.0
-189129600.0
-187920000.0
-187315200.0
-186105600.0
-185500800.0
-184896000.0
-184291200.0
-183081600.0
-181872000.0
-181267200.0
-180662400.0
-180057600.0
-179452800.0
-178848000.0
-177638400.0

In [27]:
match_df['R_Pound-for-Pound_rank'] = match_df.apply(lambda x: return_rank(x['R_fighter'],
                                                                         x['date'],
                                                                         'Pound-for-Pound'),axis=1)

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-3456000.0
-2851200.0
-1641600.0
-1036800.0
-432000.0
172800.0
-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-

-16761600.0
-16156800.0
-15552000.0
-14947200.0
-14256000.0
-13737600.0
-13132800.0
-12528000.0
-11923200.0
-11318400.0
-10713600.0
-10108800.0
-9504000.0
-8899200.0
-8294400.0
-7689600.0
-7084800.0
-6480000.0
-5875200.0
-3456000.0
-2851200.0
-1641600.0
-1036800.0
-432000.0
172800.0
-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-

In [28]:
match_df['B_Women\'s Flyweight_rank'] = match_df.apply(lambda x: return_rank(x['B_fighter'],
                                                                         x['date'],
                                                                         'Women\'s Flyweight'),axis=1)

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-109123200.0
-108518400.0
-108086400.0
-106876800.0
-105667200.0
-105062400.0
-104284800.0
-103852800.0
-103248000.0
-102470400.0
-101865600.0
-101260800.0
-100828800.0
-99619200.0
-98236800.0
-97632000.0
-97200000.0
-96422400.0
-95990400.0
-95212800.0
-94780800.0
-94176000.0
-92793600.0
-90979200.0
-90374400.0
-89942400.0
-87955200.0
-87350400.0
-86745600.0
-86140800.0
-85536000.0
-84412800.0
-83116800.0
-82684800.0
-81907200.0
-81302400.0
-80870400.0
-78883200.0
-77673600.0
-77068800.0
-76636800.0
-75427200.0
-73440000.0
-72835200.0
-72403200.0
-71798400.0
-71020800.0
-70588800.0
-69811200.0
-69379200.0
-68083200.0
-66355200.0
-65145600.0
-64540800.0
-63849600.0
-63331200.0
-62640000.0
-62035200.0
-61344000.0
-60825600.0
-60307200.0
-60134400.0
-59097600.0
-58492800.0
-57801600.0
-57283200.0
-56678400.0
-55987200.0
-54864000.0
-51840000.0
-51235200.0
-50630400.0
-50025600.0
-49420800.0
-48816000.0
-48211200.0
-47001600.0
-46396800.0
-45792000.0
-45187200.0
-44582400.0
-43977600.0
-43

-13737600.0
-13132800.0
-12528000.0
-11923200.0
-11318400.0
-10713600.0
-10108800.0
-9504000.0
-8899200.0
-8294400.0
-7689600.0
-7084800.0
-6480000.0
-5875200.0
-3456000.0
-2851200.0
-1641600.0
-1036800.0
-432000.0
172800.0
-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-20848320

In [29]:
match_df['B_Women\'s Featherweight_rank'] = match_df.apply(lambda x: return_rank(x['B_fighter'],
                                                                         x['date'],
                                                                         'Women\'s Featherweight'),axis=1)

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-135907200.0
-135129600.0
-134697600.0
-133920000.0
-133488000.0
-132710400.0
-132278400.0
-131068800.0
-129686400.0
-128476800.0
-127872000.0
-127440000.0
-126748800.0
-125625600.0
-124848000.0
-124243200.0
-122428800.0
-121824000.0
-121219200.0
-119404800.0
-118195200.0
-117590400.0
-116985600.0
-116380800.0
-115776000.0
-114739200.0
-114048000.0
-113356800.0
-112752000.0
-112320000.0
-109900800.0
-109123200.0
-108518400.0
-108086400.0
-106876800.0
-105667200.0
-105062400.0
-104284800.0
-103852800.0
-103248000.0
-102470400.0
-101865600.0
-101260800.0
-100828800.0
-99619200.0
-98236800.0
-97632000.0
-97200000.0
-96422400.0
-95990400.0
-95212800.0
-94780800.0
-94176000.0
-92793600.0
-90979200.0
-90374400.0
-89942400.0
-87955200.0
-87350400.0
-86745600.0
-86140800.0
-85536000.0
-84412800.0
-83116800.0
-82684800.0
-81907200.0
-81302400.0
-80870400.0
-78883200.0
-77673600.0
-77068800.0
-76636800.0
-75427200.0
-73440000.0
-72835200.0
-72403200.0
-71798400.0
-71020800.0
-70588800.0
-6981120

-132710400.0
-132278400.0
-131068800.0
-129686400.0
-128476800.0
-127872000.0
-127440000.0
-126748800.0
-125625600.0
-124848000.0
-124243200.0
-122428800.0
-121824000.0
-121219200.0
-119404800.0
-118195200.0
-117590400.0
-116985600.0
-116380800.0
-115776000.0
-114739200.0
-114048000.0
-113356800.0
-112752000.0
-112320000.0
-109900800.0
-109123200.0
-108518400.0
-108086400.0
-106876800.0
-105667200.0
-105062400.0
-104284800.0
-103852800.0
-103248000.0
-102470400.0
-101865600.0
-101260800.0
-100828800.0
-99619200.0
-98236800.0
-97632000.0
-97200000.0
-96422400.0
-95990400.0
-95212800.0
-94780800.0
-94176000.0
-92793600.0
-90979200.0
-90374400.0
-89942400.0
-87955200.0
-87350400.0
-86745600.0
-86140800.0
-85536000.0
-84412800.0
-83116800.0
-82684800.0
-81907200.0
-81302400.0
-80870400.0
-78883200.0
-77673600.0
-77068800.0
-76636800.0
-75427200.0
-73440000.0
-72835200.0
-72403200.0
-71798400.0
-71020800.0
-70588800.0
-69811200.0
-69379200.0
-68083200.0
-66355200.0
-65145600.0
-64540800.0
-

-49420800.0
-48816000.0
-48211200.0
-47001600.0
-46396800.0
-45792000.0
-45187200.0
-44582400.0
-43977600.0
-43372800.0
-42768000.0
-42163200.0
-41558400.0
-40953600.0
-40348800.0
-38448000.0
-37929600.0
-37238400.0
-34905600.0
-34300800.0
-33091200.0
-32486400.0
-31881600.0
-31276800.0
-30672000.0
-30067200.0
-25228800.0
-24537600.0
-23328000.0
-22809600.0
-22204800.0
-21600000.0
-20995200.0
-19699200.0
-18576000.0
-17971200.0
-17366400.0
-16761600.0
-16156800.0
-15552000.0
-14947200.0
-14256000.0
-13737600.0
-13132800.0
-12528000.0
-11923200.0
-11318400.0
-10713600.0
-10108800.0
-9504000.0
-8899200.0
-8294400.0
-7689600.0
-7084800.0
-6480000.0
-5875200.0
-3456000.0
-2851200.0
-1641600.0
-1036800.0
-432000.0
172800.0


In [30]:
match_df['B_Women\'s Strawweight_rank'] = match_df.apply(lambda x: return_rank(x['B_fighter'],
                                                                         x['date'],
                                                                         'Women\'s Strawweight'),axis=1)

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-30067200.0
-25228800.0
-24537600.0
-23328000.0
-22809600.0
-22204800.0
-21600000.0
-20995200.0
-19699200.0
-18576000.0
-17971200.0
-17366400.0
-16761600.0
-16156800.0
-15552000.0
-14947200.0
-14256000.0
-13737600.0
-13132800.0
-12528000.0
-11923200.0
-11318400.0
-10713600.0
-10108800.0
-9504000.0
-8899200.0
-8294400.0
-7689600.0
-7084800.0
-6480000.0
-5875200.0
-3456000.0
-2851200.0
-1641600.0
-1036800.0
-432000.0
172800.0
-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0


In [31]:
match_df['B_Women\'s Bantamweight_rank'] = match_df.apply(lambda x: return_rank(x['B_fighter'],
                                                                         x['date'],
                                                                         'Women\'s Bantamweight'),axis=1)

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-63331200.0
-62640000.0
-62035200.0
-61344000.0
-60825600.0
-60307200.0
-60134400.0
-59097600.0
-58492800.0
-57801600.0
-57283200.0
-56678400.0
-55987200.0
-54864000.0
-51840000.0
-51235200.0
-50630400.0
-50025600.0
-49420800.0
-48816000.0
-48211200.0
-47001600.0
-46396800.0
-45792000.0
-45187200.0
-44582400.0
-43977600.0
-43372800.0
-42768000.0
-42163200.0
-41558400.0
-40953600.0
-40348800.0
-38448000.0
-37929600.0
-37238400.0
-34905600.0
-34300800.0
-33091200.0
-32486400.0
-31881600.0
-31276800.0
-30672000.0
-30067200.0
-25228800.0
-24537600.0
-23328000.0
-22809600.0
-22204800.0
-21600000.0
-20995200.0
-19699200.0
-18576000.0
-17971200.0
-17366400.0
-16761600.0
-16156800.0
-15552000.0
-14947200.0
-14256000.0
-13737600.0
-13132800.0
-12528000.0
-11923200.0
-11318400.0
-10713600.0
-10108800.0
-9504000.0
-8899200.0
-8294400.0
-7689600.0
-7084800.0
-6480000.0
-5875200.0
-3456000.0
-2851200.0
-1641600.0
-1036800.0
-432000.0
172800.0
-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252

-30672000.0
-30067200.0
-25228800.0
-24537600.0
-23328000.0
-22809600.0
-22204800.0
-21600000.0
-20995200.0
-19699200.0
-18576000.0
-17971200.0
-17366400.0
-16761600.0
-16156800.0
-15552000.0
-14947200.0
-14256000.0
-13737600.0
-13132800.0
-12528000.0
-11923200.0
-11318400.0
-10713600.0
-10108800.0
-9504000.0
-8899200.0
-8294400.0
-7689600.0
-7084800.0
-6480000.0
-5875200.0
-3456000.0
-2851200.0
-1641600.0
-1036800.0
-432000.0
172800.0
-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-

In [32]:
match_df['B_Heavyweight_rank'] = match_df.apply(lambda x: return_rank(x['B_fighter'],
                                                                         x['date'],
                                                                         'Heavyweight'),axis=1)

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0
-193968000.0
-192672000.0
-192153600.0
-191548800.0
-190339200.0
-189648000.0
-189129600.0
-187920000.0
-187315200.0
-186105600.0
-185500800.0
-184896000.0
-184291200.0
-183081600.0

-149212800.0
-148003200.0
-146188800.0
-145843200.0
-144806400.0
-144201600.0
-143683200.0
-142387200.0
-141782400.0
-141350400.0
-140572800.0
-139968000.0
-139536000.0
-138758400.0
-138153600.0
-135907200.0
-135129600.0
-134697600.0
-133920000.0
-133488000.0
-132710400.0
-132278400.0
-131068800.0
-129686400.0
-128476800.0
-127872000.0
-127440000.0
-126748800.0
-125625600.0
-124848000.0
-124243200.0
-122428800.0
-121824000.0
-121219200.0
-119404800.0
-118195200.0
-117590400.0
-116985600.0
-116380800.0
-115776000.0
-114739200.0
-114048000.0
-113356800.0
-112752000.0
-112320000.0
-109900800.0
-109123200.0
-108518400.0
-108086400.0
-106876800.0
-105667200.0
-105062400.0
-104284800.0
-103852800.0
-103248000.0
-102470400.0
-101865600.0
-101260800.0
-100828800.0
-99619200.0
-98236800.0
-97632000.0
-97200000.0
-96422400.0
-95990400.0
-95212800.0
-94780800.0
-94176000.0
-92793600.0
-90979200.0
-90374400.0
-89942400.0
-87955200.0
-87350400.0
-86745600.0
-86140800.0
-85536000.0
-84412800.0
-8311

In [33]:
match_df['B_Light Heavyweight_rank'] = match_df.apply(lambda x: return_rank(x['B_fighter'],
                                                                         x['date'],
                                                                         'Light Heavyweight'),axis=1)

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-54864000.0
-51840000.0
-51235200.0
-50630400.0
-50025600.0
-49420800.0
-48816000.0
-48211200.0
-47001600.0
-46396800.0
-45792000.0
-45187200.0
-44582400.0
-43977600.0
-43372800.0
-42768000.0
-42163200.0
-41558400.0
-40953600.0
-40348800.0
-38448000.0
-37929600.0
-37238400.0
-34905600.0
-34300800.0
-33091200.0
-32486400.0
-31881600.0
-31276800.0
-30672000.0
-30067200.0
-25228800.0
-24537600.0
-23328000.0
-22809600.0
-22204800.0
-21600000.0
-20995200.0
-19699200.0
-18576000.0
-17971200.0
-17366400.0
-16761600.0
-16156800.0
-15552000.0
-14947200.0
-14256000.0
-13737600.0
-13132800.0
-12528000.0
-11923200.0
-11318400.0
-10713600.0
-10108800.0
-9504000.0
-8899200.0
-8294400.0
-7689600.0
-7084800.0
-6480000.0
-5875200.0
-3456000.0
-2851200.0
-1641600.0
-1036800.0
-432000.0
172800.0
-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237

-151027200.0
-149731200.0
-149212800.0
-148003200.0
-146188800.0
-145843200.0
-144806400.0
-144201600.0
-143683200.0
-142387200.0
-141782400.0
-141350400.0
-140572800.0
-139968000.0
-139536000.0
-138758400.0
-138153600.0
-135907200.0
-135129600.0
-134697600.0
-133920000.0
-133488000.0
-132710400.0
-132278400.0
-131068800.0
-129686400.0
-128476800.0
-127872000.0
-127440000.0
-126748800.0
-125625600.0
-124848000.0
-124243200.0
-122428800.0
-121824000.0
-121219200.0
-119404800.0
-118195200.0
-117590400.0
-116985600.0
-116380800.0
-115776000.0
-114739200.0
-114048000.0
-113356800.0
-112752000.0
-112320000.0
-109900800.0
-109123200.0
-108518400.0
-108086400.0
-106876800.0
-105667200.0
-105062400.0
-104284800.0
-103852800.0
-103248000.0
-102470400.0
-101865600.0
-101260800.0
-100828800.0
-99619200.0
-98236800.0
-97632000.0
-97200000.0
-96422400.0
-95990400.0
-95212800.0
-94780800.0
-94176000.0
-92793600.0
-90979200.0
-90374400.0
-89942400.0
-87955200.0
-87350400.0
-86745600.0
-86140800.0
-85

In [34]:
match_df['B_Middleweight_rank'] = match_df.apply(lambda x: return_rank(x['B_fighter'],
                                                                         x['date'],
                                                                         'Middleweight'),axis=1)

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-148003200.0
-146188800.0
-145843200.0
-144806400.0
-144201600.0
-143683200.0
-142387200.0
-141782400.0
-141350400.0
-140572800.0
-139968000.0
-139536000.0
-138758400.0
-138153600.0
-135907200.0
-135129600.0
-134697600.0
-133920000.0
-133488000.0
-132710400.0
-132278400.0
-131068800.0
-129686400.0
-128476800.0
-127872000.0
-127440000.0
-126748800.0
-125625600.0
-124848000.0
-124243200.0
-122428800.0
-121824000.0
-121219200.0
-119404800.0
-118195200.0
-117590400.0
-116985600.0
-116380800.0
-115776000.0
-114739200.0
-114048000.0
-113356800.0
-112752000.0
-112320000.0
-109900800.0
-109123200.0
-108518400.0
-108086400.0
-106876800.0
-105667200.0
-105062400.0
-104284800.0
-103852800.0
-103248000.0
-102470400.0
-101865600.0
-101260800.0
-100828800.0
-99619200.0
-98236800.0
-97632000.0
-97200000.0
-96422400.0
-95990400.0
-95212800.0
-94780800.0
-94176000.0
-92793600.0
-90979200.0
-90374400.0
-89942400.0
-87955200.0
-87350400.0
-86745600.0
-86140800.0
-85536000.0
-84412800.0
-83116800.0
-82684

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

In [35]:
match_df['B_Welterweight_rank'] = match_df.apply(lambda x: return_rank(x['B_fighter'],
                                                                         x['date'],
                                                                         'Welterweight'),axis=1)

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-46396800.0
-45792000.0
-45187200.0
-44582400.0
-43977600.0
-43372800.0
-42768000.0
-42163200.0
-41558400.0
-40953600.0
-40348800.0
-38448000.0
-37929600.0
-37238400.0
-34905600.0
-34300800.0
-33091200.0
-32486400.0
-31881600.0
-31276800.0
-30672000.0
-30067200.0
-25228800.0
-24537600.0
-23328000.0
-22809600.0
-22204800.0
-21600000.0
-20995200.0
-19699200.0
-18576000.0
-17971200.0
-17366400.0
-16761600.0
-16156800.0
-15552000.0
-14947200.0
-14256000.0
-13737600.0
-13132800.0
-12528000.0
-11923200.0
-11318400.0
-10713600.0
-10108800.0
-9504000.0
-8899200.0
-8294400.0
-7689600.0
-7084800.0
-6480000.0
-5875200.0
-3456000.0
-2851200.0
-1641600.0
-1036800.0
-432000.0
172800.0
-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-2305152

In [36]:
match_df['B_Lightweight_rank'] = match_df.apply(lambda x: return_rank(x['B_fighter'],
                                                                         x['date'],
                                                                         'Lightweight'),axis=1)

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-94176000.0
-92793600.0
-90979200.0
-90374400.0
-89942400.0
-87955200.0
-87350400.0
-86745600.0
-86140800.0
-85536000.0
-84412800.0
-83116800.0
-82684800.0
-81907200.0
-81302400.0
-80870400.0
-78883200.0
-77673600.0
-77068800.0
-76636800.0
-75427200.0
-73440000.0
-72835200.0
-72403200.0
-71798400.0
-71020800.0
-70588800.0
-69811200.0
-69379200.0
-68083200.0
-66355200.0
-65145600.0
-64540800.0
-63849600.0
-63331200.0
-62640000.0
-62035200.0
-61344000.0
-60825600.0
-60307200.0
-60134400.0
-59097600.0
-58492800.0
-57801600.0
-57283200.0
-56678400.0
-55987200.0
-54864000.0
-51840000.0
-51235200.0
-50630400.0
-50025600.0
-49420800.0
-48816000.0
-48211200.0
-47001600.0
-46396800.0
-45792000.0
-45187200.0
-44582400.0
-43977600.0
-43372800.0
-42768000.0
-42163200.0
-41558400.0
-40953600.0
-40348800.0
-38448000.0
-37929600.0
-37238400.0
-34905600.0
-34300800.0
-33091200.0
-32486400.0
-31881600.0
-31276800.0
-30672000.0
-30067200.0
-25228800.0
-24537600.0
-23328000.0
-22809600.0
-22204800.0
-216

-142387200.0
-141782400.0
-141350400.0
-140572800.0
-139968000.0
-139536000.0
-138758400.0
-138153600.0
-135907200.0
-135129600.0
-134697600.0
-133920000.0
-133488000.0
-132710400.0
-132278400.0
-131068800.0
-129686400.0
-128476800.0
-127872000.0
-127440000.0
-126748800.0
-125625600.0
-124848000.0
-124243200.0
-122428800.0
-121824000.0
-121219200.0
-119404800.0
-118195200.0
-117590400.0
-116985600.0
-116380800.0
-115776000.0
-114739200.0
-114048000.0
-113356800.0
-112752000.0
-112320000.0
-109900800.0
-109123200.0
-108518400.0
-108086400.0
-106876800.0
-105667200.0
-105062400.0
-104284800.0
-103852800.0
-103248000.0
-102470400.0
-101865600.0
-101260800.0
-100828800.0
-99619200.0
-98236800.0
-97632000.0
-97200000.0
-96422400.0
-95990400.0
-95212800.0
-94780800.0
-94176000.0
-92793600.0
-90979200.0
-90374400.0
-89942400.0
-87955200.0
-87350400.0
-86745600.0
-86140800.0
-85536000.0
-84412800.0
-83116800.0
-82684800.0
-81907200.0
-81302400.0
-80870400.0
-78883200.0
-77673600.0
-77068800.0


In [37]:
match_df['B_Featherweight_rank'] = match_df.apply(lambda x: return_rank(x['B_fighter'],
                                                                         x['date'],
                                                                         'Featherweight'),axis=1)

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-30672000.0
-30067200.0
-25228800.0
-24537600.0
-23328000.0
-22809600.0
-22204800.0
-21600000.0
-20995200.0
-19699200.0
-18576000.0
-17971200.0
-17366400.0
-16761600.0
-16156800.0
-15552000.0
-14947200.0
-14256000.0
-13737600.0
-13132800.0
-12528000.0
-11923200.0
-11318400.0
-10713600.0
-10108800.0
-9504000.0
-8899200.0
-8294400.0
-7689600.0
-7084800.0
-6480000.0
-5875200.0
-3456000.0
-2851200.0
-1641600.0
-1036800.0
-432000.0
172800.0
-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-

-11318400.0
-10713600.0
-10108800.0
-9504000.0
-8899200.0
-8294400.0
-7689600.0
-7084800.0
-6480000.0
-5875200.0
-3456000.0
-2851200.0
-1641600.0
-1036800.0
-432000.0
172800.0
-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-2048

In [38]:
match_df['B_Bantamweight_rank'] = match_df.apply(lambda x: return_rank(x['B_fighter'],
                                                                         x['date'],
                                                                         'Bantamweight'),axis=1)

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

9
-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

In [39]:
match_df['B_Flyweight_rank'] = match_df.apply(lambda x: return_rank(x['B_fighter'],
                                                                         x['date'],
                                                                         'Flyweight'),axis=1)

-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0
-193968000.0
-192672000.0
-192153600.0
-191548800.0
-190339200.0
-189648000.0
-189129600.0
-187920000.0
-187315200.0
-186105600.0
-185500800.0
-184896000.0
-184291200.0
-183081600.0
-181872000.0
-181267200.0
-180662400.0
-180057600.0
-179452800.0
-178848000.0
-177638400.0

-57283200.0
-56678400.0
-55987200.0
-54864000.0
-51840000.0
-51235200.0
-50630400.0
-50025600.0
-49420800.0
-48816000.0
-48211200.0
-47001600.0
-46396800.0
-45792000.0
-45187200.0
-44582400.0
-43977600.0
-43372800.0
-42768000.0
-42163200.0
-41558400.0
-40953600.0
-40348800.0
-38448000.0
-37929600.0
-37238400.0
-34905600.0
-34300800.0
-33091200.0
-32486400.0
-31881600.0
-31276800.0
-30672000.0
-30067200.0
-25228800.0
-24537600.0
-23328000.0
-22809600.0
-22204800.0
-21600000.0
-20995200.0
-19699200.0
-18576000.0
-17971200.0
-17366400.0
-16761600.0
-16156800.0
-15552000.0
-14947200.0
-14256000.0
-13737600.0
-13132800.0
-12528000.0
-11923200.0
-11318400.0
-10713600.0
-10108800.0
-9504000.0
-8899200.0
-8294400.0
-7689600.0
-7084800.0
-6480000.0
-5875200.0
-3456000.0
-2851200.0
-1641600.0
-1036800.0
-432000.0
172800.0
-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142

In [40]:
match_df['B_Pound-for-Pound_rank'] = match_df.apply(lambda x: return_rank(x['B_fighter'],
                                                                         x['date'],
                                                                         'Pound-for-Pound'),axis=1)



-254448000.0
-253843200.0
-253238400.0
-252633600.0
-252028800.0
-250819200.0
-249004800.0
-248400000.0
-247795200.0
-247190400.0
-244684800.0
-243561600.0
-242870400.0
-241142400.0
-239328000.0
-238723200.0
-237513600.0
-236563200.0
-236304000.0
-235958400.0
-234489600.0
-232934400.0
-232070400.0
-231465600.0
-230515200.0
-230256000.0
-229651200.0
-228355200.0
-227836800.0
-227232000.0
-226022400.0
-225331200.0
-224467200.0
-223603200.0
-222998400.0
-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0

-101260800.0
-100828800.0
-99619200.0
-98236800.0
-97632000.0
-97200000.0
-96422400.0
-95990400.0
-95212800.0
-94780800.0
-94176000.0
-92793600.0
-90979200.0
-90374400.0
-89942400.0
-87955200.0
-87350400.0
-86745600.0
-86140800.0
-85536000.0
-84412800.0
-83116800.0
-82684800.0
-81907200.0
-81302400.0
-80870400.0
-78883200.0
-77673600.0
-77068800.0
-76636800.0
-75427200.0
-73440000.0
-72835200.0
-72403200.0
-71798400.0
-71020800.0
-70588800.0
-69811200.0
-69379200.0
-68083200.0
-66355200.0
-65145600.0
-64540800.0
-63849600.0
-63331200.0
-62640000.0
-62035200.0
-61344000.0
-60825600.0
-60307200.0
-60134400.0
-59097600.0
-58492800.0
-57801600.0
-57283200.0
-56678400.0
-55987200.0
-54864000.0
-51840000.0
-51235200.0
-50630400.0
-50025600.0
-49420800.0
-48816000.0
-48211200.0
-47001600.0
-46396800.0
-45792000.0
-45187200.0
-44582400.0
-43977600.0
-43372800.0
-42768000.0
-42163200.0
-41558400.0
-40953600.0
-40348800.0
-38448000.0
-37929600.0
-37238400.0
-34905600.0
-34300800.0
-33091200.0
-3

-221788800.0
-221702400.0
-221184000.0
-221097600.0
-220579200.0
-220492800.0
-219974400.0
-219369600.0
-218764800.0
-218678400.0
-216950400.0
-216604800.0
-216345600.0
-215740800.0
-215136000.0
-215049600.0
-214531200.0
-213321600.0
-213148800.0
-212716800.0
-212112000.0
-211507200.0
-210297600.0
-209606400.0
-208483200.0
-207878400.0
-206064000.0
-205459200.0
-204854400.0
-204249600.0
-204163200.0
-203644800.0
-203040000.0
-202435200.0
-201830400.0
-200016000.0
-198806400.0
-198201600.0
-197596800.0
-196387200.0
-195782400.0
-195177600.0
-193968000.0
-192672000.0
-192153600.0
-191548800.0
-190339200.0
-189648000.0
-189129600.0
-187920000.0
-187315200.0
-186105600.0
-185500800.0
-184896000.0
-184291200.0
-183081600.0
-181872000.0
-181267200.0
-180662400.0
-180057600.0
-179452800.0
-178848000.0
-177638400.0
-177033600.0
-176428800.0
-175824000.0
-175219200.0
-173923200.0
-172800000.0
-170985600.0
-170380800.0
-168566400.0
-167356800.0
-166752000.0
-166147200.0
-165542400.0
-164332800.0

In [41]:
def return_better_rank(r_rank, b_rank):
    if (r_rank == ''):
        if b_rank != '':
            return('Blue')
        else:
            return('neither')
    if (b_rank == ''):
        return('Red')
    r_rank = int(r_rank)
    b_rank = int(b_rank)
    if (r_rank < b_rank):
        return('Red')
    else:
        return('Blue')
    return('neither')

In [42]:
match_df['better_rank'] = match_df.apply(lambda x: return_better_rank(x['R_match_weightclass_rank'],
                                                                         x['B_match_weightclass_rank']),axis=1)

In [43]:
display(match_df.head())

R_fighter         B_fighter  R_odds  B_odds  R_ev  B_ev  \
0  Jairzinho Rozenstruik        Ciryl Gane     NaN     NaN   NaN   NaN   
1          Nikita Krylov  Magomed Ankalaev     NaN     NaN   NaN   NaN   
3           Pedro Munhoz     Jimmie Rivera     NaN     NaN   NaN   NaN   
4           Alex Caceres       Kevin Croom     NaN     NaN   NaN   NaN   
5    Alexander Hernandez     Thiago Moises     NaN     NaN   NaN   NaN   

        date                location country Winner  ...  B_td_attempted_bout  \
0 2021-02-27  Las Vegas, Nevada, USA     USA   Blue  ...                  NaN   
1 2021-02-27  Las Vegas, Nevada, USA     USA   Blue  ...                  NaN   
3 2021-02-27  Las Vegas, Nevada, USA     USA    Red  ...                  NaN   
4 2021-02-27  Las Vegas, Nevada, USA     USA    Red  ...                  NaN   
5 2021-02-27  Las Vegas, Nevada, USA     USA   Blue  ...                  NaN   

  R_td_pct_bout B_td_pct_bout  R_sub_attempts_bout  B_sub_attempts_bout  \
0           NaN           NaN                  NaN                  NaN   
1           NaN           NaN                  NaN                  NaN   
3           NaN           NaN                  NaN                  NaN   
4           NaN           NaN                  NaN                  NaN   
5           NaN           NaN                  NaN                  NaN   

   R_pass_bout  B_pass_bout  R_rev_bout  B_rev_bout  label  
0          NaN          NaN         NaN         NaN      1  
1          NaN          NaN         NaN         NaN      1  
3          NaN          NaN         NaN         NaN      0  
4          NaN          NaN         NaN         NaN      0  
5          NaN          NaN         NaN         NaN      1  

[5 rows x 138 columns]

In [44]:
#test = (match_df.iloc[1384])

In [45]:
#display(test[['R_fighter', 'R_match_weightclass_rank', 'B_fighter', 'B_match_weightclass_rank', 'date', 'better_rank']])


In [46]:
match_df.drop(columns=['label'], inplace=True)
match_df.to_csv('scraped_event_with_ranks.csv')

### Take a quick look at how the better ranked fighter does:

In [47]:
"""temp_df = match_df[match_df['better_rank']=='Red'].copy()
red_favorite_count = (len(temp_df))
temp_df = temp_df[temp_df['Winner']=='Red']
red_winner_count = len(temp_df)

red_pct = (red_winner_count / red_favorite_count)

temp_df = match_df[match_df['better_rank']=='Blue'].copy()
blue_favorite_count = (len(temp_df))
temp_df = temp_df[temp_df['Winner']=='Blue']
blue_winner_count = len(temp_df)

blue_pct = (blue_winner_count / blue_favorite_count)
print('When Red has the better rank they win ', "{:.2f}".format(red_pct*100), '% of the time')
print('When Blue has the better rank they win ', "{:.2f}".format(blue_pct*100), '% of the time')
"""

'temp_df = match_df[match_df[\'better_rank\']==\'Red\'].copy()\nred_favorite_count = (len(temp_df))\ntemp_df = temp_df[temp_df[\'Winner\']==\'Red\']\nred_winner_count = len(temp_df)\n\nred_pct = (red_winner_count / red_favorite_count)\n\ntemp_df = match_df[match_df[\'better_rank\']==\'Blue\'].copy()\nblue_favorite_count = (len(temp_df))\ntemp_df = temp_df[temp_df[\'Winner\']==\'Blue\']\nblue_winner_count = len(temp_df)\n\nblue_pct = (blue_winner_count / blue_favorite_count)\nprint(\'When Red has the better rank they win \', "{:.2f}".format(red_pct*100), \'% of the time\')\nprint(\'When Blue has the better rank they win \', "{:.2f}".format(blue_pct*100), \'% of the time\')\n'

In [48]:
#print(blue_favorite_count)